# 问题一：列表和元组的内部实现

有关列表（list）和元组（tuple）的内部实现，想知道里边是 linked list 或 array，还是把 array linked 这样的方式？关于这个问题，我们可以分别从源码来看。先来看 Python 3.7 的 list 源码。你可以先自己阅读下面两个链接里的内容。
- [listobject.h](https://github.com/python/cpython/blob/949fe976d5c62ae63ed505ecf729f815d0baccfc/Include/listobject.h#L23)
- [listobject.c](https://github.com/python/cpython/blob/3d75bd15ac82575967db367c517d7e6e703a6de3/Objects/listobject.c#L33)

我把 list 的具体结构
![media/list.png](media/list.png)

可以看到，`list` 本质上是一个 `over-allocate` 的 `array`。其中，`ob_item` 是一个指针列表，里面的每一个指针都指向列表的元素。而 `allocated` 则存储了这个列表已经被分配的空间大小。

需要注意的是，`allocated` 与列表实际空间大小的区别。列表实际空间大小，是指 `len(list)` 返回的结果，即上述代码注释中的 `ob_size`，表示这个列表总共存储了多少个元素。实际情况下，为了优化存储结构，避免每次增加元素都要重新分配内存，列表预分配的空间 `allocated` 往往会大于 `ob_size`。所以，它们的关系为：`allocated >= len(list) = ob_size`。如果当前列表分配的空间已满（即 allocated == len(list)），则会向系统请求更大的内存空间，并把原来的元素全部拷贝过去。列表每次分配空间的大小，遵循下面的模式：
```
0, 4, 8, 16, 25, 35, 46, 58, 72, 88, ...
```

我们再来分析元组。下面是 Python 3.7 的 tuple 源码，同样的，你可以先自己阅读一下。
- [tupleobject.h](https://github.com/python/cpython/blob/3d75bd15ac82575967db367c517d7e6e703a6de3/Include/tupleobject.h#L25)
- [tupleobject.c](https://github.com/python/cpython/blob/3d75bd15ac82575967db367c517d7e6e703a6de3/Objects/tupleobject.c#L16)

同样的，下面为 tuple 的具体结构：
![](media/tuple.png)

你可以看到，它和 `list` 相似，本质也是一个 `array`，但**是空间大小固定**。不同于一般 `array`，`Python` 的 `tuple` 做了许多优化，来提升在程序中的效率。举个例子，当 `tuple` 的大小不超过 `20` 时，Python 就会把它缓存在内部的一个 `free list` 中。这样，如果你以后需要再去创建同样的 `tuple`，Python 就可以直接从缓存中载入，提高了程序运行效率

# 问题二：为什么在旧哈希表中，元素会越来越稀疏？

你会发现，它是一个 `over-allocate` 的 `array`，根据元素键（key）的哈希值，来计算其应该被插入位置的索引。因此，假设我有下面这样一个字典：`{'name': 'mike', 'dob': '1999-01-01', 'gender': 'male'}`，那么这个字典便会存储为类似下面的形式：

这里的 `'---'`，表示**这个位置没有元素**，但是已经分配了内存。我们知道，当哈希表剩余空间小于 1/3 时，为了保证相关操作的高效性并避免哈希冲突，就会重新分配更大的内存。所以，当哈希表中的元素越来越多时，分配了内存但里面没有元素的位置，也会变得越来越多。这样一来，哈希表便会越来越稀疏。而新哈希表的结构，改变了这一点，也大大提高了空间的利用率。新哈希表的结构如下所示：

可以看到，它把存储结构分成了 `Indices` 和 `Entries` 这两个 `array`，而 `None` 代表这个位置**分配了内存但没有元素**。我们同样还用上面这个例子，它在新哈希表中的存储模式，就会变为下面这样：`

其中，`Indices` 中元素的值，对应 `entries` 中相应的索引。比如 `indices` 中的 `1`，就对应着 `entries[1]` ，即 `'dob': '1999-01-01'` 。对比之下，我们会清晰感受到，新哈希表中的空间利用率，相比于旧哈希表有大大的提升。

# 问题三：有关异常的困扰

![](media/error.png)

这个问题其实有点 `tricky`，如果你查阅官方文档，会看到这么一句话 `When an exception has been assigned using as target, it is cleared at the end of the except clause. ` 这句话意思是，如果你在异常处理的 `except block` 中，把异常赋予了一个变量，那么这个变量会在 `except block` 执行结束时被删除，相当于下面这样的表示：

In [2]:
e = 1
try:
    1 / 0
except ZeroDivisionError as e:
    try:
        pass
    finally:
        del e

这里的 `e` 一开始指向整数 `1`，但是在 `except block` 结束时被删除了（`del e`），所以程序执行就会抛出 `NameError` 的异常。因此，这里提醒我们，在平时写代码时，一定要保证 `except` 中异常赋予的变量，在之后的语句中不再被用到。

# 问题四：关于多态和全局变量的修改

- `Python` 自己判断类型的多态和子类继承的多态 `Polymorphism` 是否相同？
- 函数内部不能直接用 `+=` 等修改全局变量，但是对于 `list` 全局变量，却可以使用 `append`、`extend` 之类修改，这是为什么呢?

对于第一个问题，要搞清楚多态的概念，**多态是指有多种不同的形式**。因此，判断类型的多态和子类继承的多态，在本质上都是一样的，只不过你可以把它们理解为多态的两种不同表现。

再来看第二个问题。当全局变量指向的对象不可变时，比如是整型、字符串等等，如果你尝试在函数内部改变它的值，却不加关键字 global，就会抛出异常：`

In [3]:
x = 1

def func():
    x += 1
func()
x

UnboundLocalError: local variable 'x' referenced before assignment

这是因为，程序默认函数内部的 `x` 是局部变量，而你没有为其赋值就直接引用，显然是不可行。不过，如果**全局变量指向的对象是可变的**，比如是列表、字典等等，你就可以在函数内部修改它了：

In [5]:
x = [1]

def func():
    x.append(2)
    
func()
x


[1, 2]

当然，需要注意的是，这里的 `x.append(2)`，并没有改变变量 `x`，`x` 依然指向原来的列表。事实上，这句话的意思是，**访问 `x` 指向的列表，并在这个列表的末尾增加 2。**